In [19]:
#testget_gnd_state_probs_and_approx_ratio.ipynb
from pyaqc.RCModules.aqc_rigetti import get_gnd_state_probs_and_approx_ratio
#ONE AMBULANCE adjacency table
from pyaqc.RCModules.PlatformConversion import print_QUBOdetails, CreateAmbulanceAdjacency,distance #   qc_all_env and rigoct38_env
import numpy as np 
import unicodedata as ud
betagreek =  ud.lookup('GREEK SMALL LETTER BETA')
gammagreek =  ud.lookup('GREEK SMALL LETTER gamma')

################  Set up type of problem to solve ################
n_destinations = 11
n_qubits = n_destinations
HammingWeightOfConstraint =  n_qubits - 1

gridWidth = n_qubits
ConstraintMultiplier = 140             # I have used 14 for 5 qubit with constraints and traditional mixer, 140 for q11, tim 200
remove_constraint=  1   # remove the constraints when using an XY mixer
select_qubo_model = 1   # we nearly always use this
################ Create 1) 'Adjacency' table of the problem, and 2) 'Adjacency_constraint' of the problems constraints 3) Adj_x_mixer_type_constraints ################
#'Adjacency' table of the problem
Adjacency = CreateAmbulanceAdjacency(   
    gridWidth,
    n_qubits = n_qubits, 
    ConstraintMultiplier = ConstraintMultiplier, 
    Adddistance = 1,
    remove_constraint= remove_constraint, 
    HammingWeightOfConstraint= HammingWeightOfConstraint,
    qubo_model=select_qubo_model)
#'Adjacency_constraint' of the problems constraints
Adjacency_constraint = CreateAmbulanceAdjacency(   
    gridWidth,
    n_qubits = n_qubits, 
    ConstraintMultiplier = ConstraintMultiplier, 
    Adddistance = 0,
    remove_constraint= remove_constraint, 
    HammingWeightOfConstraint= HammingWeightOfConstraint,
    qubo_model=select_qubo_model)
Adj_x_mixer_type_constraints = CreateAmbulanceAdjacency(   
    gridWidth,
    n_qubits = n_qubits, 
    ConstraintMultiplier = ConstraintMultiplier, 
    Adddistance = 0,
    remove_constraint= 0, 
    HammingWeightOfConstraint= HammingWeightOfConstraint,
    qubo_model=select_qubo_model)
################ ... and print ################
if select_qubo_model:print('Qubo ', end='')
else: print('Ising ', end='')
print_QUBOdetails(Adjacency,n_qubits, 'Ambulance problem')
n_ambulance=1
Ansatz_type = {'n_qubits':n_qubits, 'p':0,'Mixer':0,'prog_init':0, 'ConstraintMultiplier':0, 'remove_constraint':0, 'Optimizer_maxiter': 0}
Ansatz_type['ConstraintMultiplier'] = ConstraintMultiplier
Ansatz_type['remove_constraint'] = remove_constraint
############################# convert Adjacency to Pauli sum of observeable edges and nodes #############################
from pyaqc.RCModules.aqc_rigetti import create_ring_mixer, Adjacency_qubo_to_Regetti,Adjacency_Ising_to_Regetti,Dicke_state
ListPauli_termsMy,SumPauli_termsMy  = Adjacency_qubo_to_Regetti( Adjacency, n_qubits)

print(SumPauli_termsMy)

from pyaqc.RCModules.aqc_rigetti import get_gnd_state_probs_and_approx_ratio,ansatz_init_XYMixer,ansatz_prog_init,ansatz_EV_init
from pyaqc.RCModules.aqc_rigetti import create_ring_mixer,create_local_ring_mixer,Dicke_state , Dicke_state_local
from pyquil.paulis import *  #  exponential_map, exponentiate_commuting_pauli_sum
from pyquil.api import *
from pyquil.gates import *

def get_mixer(n_qubits,remove_constraint,n_ambulance ):
    """
    Return Rigetti observables that can be exponentiated, and a description 'Ansatz_type'
    type: Pauli Observables
    """
    MyMixerHam = []
    MyMixerHam.append(0)                # Perhaps required by QAOA()
    if not remove_constraint:           # when the extra hard constraints are in place use an X mixer, otherwise an XY mixer
        for n in range(n_qubits):       
            MyMixerHam[0] += 1*sX(n) 
        Ansatz_type['Mixer'] ='TRADITIONAL /  Farhi  mixer'
    else:
        XYmixer = create_ring_mixer(n_qubits)
        MyMixerHam[0] = XYmixer         #XY
        Ansatz_type['Mixer'] ='XY Ring mixer'
        if n_ambulance==2:
            MyMixerHam, prog_init_three_XYMixers = three_XYMixers_Dicke_states(n_qubits, n_destinations) 
            Ansatz_type['prog_init'] = 'Three Dicke states'
            Ansatz_type['Mixer'] ='Three XY Ring mixers' 
    
    if n_ambulance==2:
        labels= ['Destination XYmixer','Start0 XYmixer','Start1 XYmixer']
        for n_mixer, mixer in enumerate(MyMixerHam):
            print(labels[n_mixer])
            print(mixer)
    else: 
        print(Ansatz_type['Mixer'])
        print(MyMixerHam[0])        
        
    return MyMixerHam, Ansatz_type
def get_prog_init(n_qubits,remove_constraint,n_ambulance ):
    """
    Return the Quantum circuit of the initial state (eg Hamamard for Xmixer)
    """
    prog_init = Program()

    if not remove_constraint:
        #A) 
        print('Hadamard initial state.   ', n_qubits, ' = n_qubits')
        for n in range(n_qubits):
            prog_init += H(n)
    else:
        if n_ambulance==2:
            #B)
            #MyMixerHam, prog_init_three_XYMixers = three_XYMixers_Dicke_states(n_qubits, n_destinations)  
            prog_init = prog_init_three_XYMixers       # this is just a reminder that the calculation was made earlier in the 'Mixer used' cell
            print('The initial state is made of three separate Dicke_state s')
        else:
            #C)
            print('Dicke_state initial state',HammingWeightOfConstraint, ' = HW')
            prog_init = Dicke_state(n_qubits, HammingWeightOfConstraint)

    return prog_init

prog_init = get_prog_init(n_qubits,remove_constraint,n_ambulance )
MyMixerHam, Ansatz_type = get_mixer(n_qubits,remove_constraint,n_ambulance )

Qubo ADJACENCY MATRIX Ambulance problem  

Qubit  q0   q1   q2   q3   q4   q5   q6   q7   q8   q9  q10 

  q0   0    -1   -4   -9  -16  -25  -36  -49  -64  -81  -100

  q1             -1   -4   -9  -16  -25  -36  -49  -64  -81 

  q2                  -1   -4   -9  -16  -25  -36  -49  -64 

  q3                       -1   -4   -9  -16  -25  -36  -49 

  q4                            -1   -4   -9  -16  -25  -36 

  q5                                 -1   -4   -9  -16  -25 

  q6                                      -1   -4   -9  -16 

  q7                                           -1   -4   -9 

  q8                                                -1   -4 

  q9                                                     -1 

(-302.5+0j)*I + (-0.25+0j)*Z0*Z1 + (96.25+0j)*Z0 + (71.5+0j)*Z1 + (-1+0j)*Z0*Z2 + (52.25+0j)*Z2 + (-2.25+0j)*Z0*Z3 + (38.5+0j)*Z3 + (-4+0j)*Z0*Z4 + (30.25+0j)*Z4 + (-6.25+0j)*Z0*Z5 + (27.5+0j)*Z5 + (-9+0j)*Z0*Z6 + (30.25+0j)*Z6 + (-12.25+0j)*Z0*Z7 + (38.5+0j)*Z7 + (-16+0j)*Z

In [20]:
# code plan: find bg for a q5 1 A problem, then XY mixer, then q16 xy mixer 2 amb,
from pyaqc.RCModules.QAOARC import decimal_state_to_binary_reversed, Energy_of_binary_state
from pyaqc.RCModules.aqc_rigetti import min_energy
def get_approx_ratio_init(ansatz_prog,prog_init,ListPauli_termsMy,SumPauli_termsMy,n_qubits,p,Adjacency_constraint,Adj_x_mixer_type_constraints,  prt_details=1,state_feasible=None ):
    """

    Initialise the function, get_gnd_state_probs_and_approx_ratio(opt_betagamma), in one of two ways
        1) Each call will be for the same ising problem. Hence, it will use the same E_min_feas,  E_max_feas, and min_energy_constraint
        2) each call of 'get_gnd_state_probs_and_approx_ratio' will use a different betagamma,  otherwise a p based on opt_betagamma

    """
    
    ### Calculate min_energy_constraint, then E_min_feas and E_max_fease  by a 2^n search
    
    "' get  Minimum energy of the Adjacency by considering all 2^n_qubits states. NO this does not work as the lowest constraint energy is not a 'feasible' state "
    "state_feasible must be supplied so that the adjacency's (Adj_x_mixer_type_constraints) energy in that state can be found type:float"
    if isinstance(state_feasible,str):
        state_feasible=state_feasible.replace(' ','')
    min_energy_constraint_X_mixer = min_energy( Adj_x_mixer_type_constraints, n_qubits, state_feasible=state_feasible)      ##use problem's mixer_c
    min_energy_constraint_problem = min_energy( Adjacency_constraint, n_qubits, state_feasible=state_feasible)              ##use problem's actualmixer_c
    
    ListPauli_terms_Xconstr,SumPauli_terms_Xconstr  = Adjacency_qubo_to_Regetti( Adj_x_mixer_type_constraints, n_qubits)
#    print(Energy_of_binary_state( 27, SumPauli_terms_Xconstr, n_qubits), ' c min')
    def energy_min_max_feasible():
        seen_first_compliant_state = 0
        E_min_feas = Energy_of_binary_state( 0, SumPauli_termsMy,n_qubits)-min_energy_constraint_problem
        for n in range(2**n_qubits):
            E_feas_of_state = Energy_of_binary_state( n, SumPauli_termsMy,n_qubits)-min_energy_constraint_problem
            if Energy_of_binary_state( n, SumPauli_terms_Xconstr, n_qubits) == min_energy_constraint_X_mixer:
                if seen_first_compliant_state==0:
                    E_max_feas = E_feas_of_state
                E_max_feas = max(E_max_feas,E_feas_of_state)
                E_min_feas = min(E_min_feas,E_feas_of_state)
                seen_first_compliant_state = 1
        return E_min_feas, E_max_feas 

    E_min_feas, E_max_feas = energy_min_max_feasible()
    print(E_min_feas, E_max_feas,min_energy_constraint_X_mixer,'E_min_feas, E_max_feas, min_energy_constraint_X_mixer',min_energy_constraint_problem, ' = min_energy_constraint_problem' )
    
    def get_gnd_state_probs_and_approx_ratio(opt_betagamma,  prt_details=1):  
        """
         return A) the cumulative probability of the lowest energy state sampled, 
                B) 'approx_ratio' from the feasible states, (defined as states that minimize 'Adjacency_constraint') the average energy calculated without any constraint                       energy / ground state energy.
                        Or -1 if 'Adjacency_constraint' is not supplied
                        As defined by Wang in arXiv:1904.09314v2 [quant-ph] 21 May 2020, page 3 col 2
                C) The probability that one of the suggested states is a feasible state(ie complies with all constraint )
        type: float,float,float

        param
            ansatz_prog
                type:function that returns a Rigetti.Program()
            'Adjacency_constraint'  The adjacency table which contain only the constraint energies , and no Soft constraints (eg distance)
            This allows the constraint energies to be calculated separately from the non-constraint energies.
                type: dict example {(0, 0): 5,  (1, 1): -1,  (2, 2): -1}
            
        kwargs
            
            'state_feasible' a string of a state that complies with all the constraints in Adjacency_constraint. For example;  '11011'. When supplied the minimum energy of                 all the constraints is calculated in one calculation, rather than 2^n_qubits calculations. This speeds up the approximation ratio calculation
                type: string of a binary state
            
            
        if p == len(opt_betagamma)//2=False, the ansatz_prog is re initiated each call of get_gnd_state_probs_and_approx_ratio, to reflect the p implied by opt_betagamma (eg when creating the approx ratio for many different p s of the same problem)
        if p == len(opt_betagamma)//2=True , execution time is saved by using the ansatz_prog that was supplied by get_approx_ratio_init (eg when looking for betagamma minimum)
        In the feasible space, what is the average energy found after substituting the actual constraint energy with 0? 
        """
        prob_gnd_state = 0
        prob_feasible_state = 0
        EV_feasible = 0 
        EV_not_feasible = 0 
        n_feasible = 0
        approx_ratio = -1
        EV_all_suggestions = 0
        Expectation_of_state = 0
        
        if p == len(opt_betagamma)//2:
            # calculate solution_ansatz with latest opt_betagamma, using the same p as when get_approx_ratio_init first called
            solution_ansatz = ansatz_prog(opt_betagamma)
        else:
            # calculate solution_ansatz with latest opt_betagamma and its implied p
            p_new = len(opt_betagamma)//2
            ansatz_prog_flexible_p = ansatz_prog_init(prog_init, ListPauli_termsMy, n_qubits, n_destinations,   p_new,MyMixerHam)
            solution_ansatz = ansatz_prog_flexible_p(opt_betagamma)
            
        
        probsAbsolute  = WavefunctionSimulator().wavefunction(solution_ansatz).get_outcome_probs()
        opt_probs_absolute = [x for x in probsAbsolute.values()]
        show_debug=1
        for n in range(2**n_qubits):
        
            if opt_probs_absolute[n] > 0.00001 :    #|qn...q0>
                # calculation of energy, using n as the state, NOT recalculating the state by building its quantum circuit eg Program(X(0),I(1))
                Expectation_of_state = Energy_of_binary_state( n, SumPauli_termsMy,n_qubits)    

            if Expectation_of_state <= E_min_feas +  min_energy_constraint_problem:   
                
                #psi_opt, state_string = decimal_state_to_binary_reversed(n,n_qubits)
                prob_gnd_state += opt_probs_absolute[n]
            
            # If the Adjacency_constraint has an energy that is the minimium possible for a feasible state, the x mixer constraints have been met, ie space is 'feasible'..
            #...then the energy, without any problem_constraint energy, is averaged.
            if Energy_of_binary_state( n, SumPauli_terms_Xconstr, n_qubits) == min_energy_constraint_X_mixer: #use X mixer_c
                EV_feasible += opt_probs_absolute[n] * (Expectation_of_state - min_energy_constraint_problem )  #use problem's mixer_c
                prob_feasible_state +=opt_probs_absolute[n]
                n_feasible +=1
            elif show_debug:
                EV_not_feasible += opt_probs_absolute[n] * Expectation_of_state 
            if show_debug:EV_all_suggestions += opt_probs_absolute[n] * Expectation_of_state 
        if show_debug:
            approx_ratio_old = EV_feasible/(prob_feasible_state*E_min_feas)
            print(approx_ratio_old, ' = approx_ratio_old', EV_feasible/prob_feasible_state, EV_not_feasible,EV_all_suggestions)

        approx_ratio = (EV_feasible/prob_feasible_state-E_max_feas) /(E_min_feas-E_max_feas)
        return prob_gnd_state, approx_ratio, prob_feasible_state
    return get_gnd_state_probs_and_approx_ratio

q5p1TradXmHqq = [6.898, -0.022] # nm -252.8 #11.1% Energy_max =-20 , Energy_min = -40 when constraint are complied with and have zero energy contribution to the cost function
q5p1RmDi = [3.9924273766343394, 2.827450565142573] # -38.4 EV of problem   71.1% r_approx new = 92.1%
q11p1RmDi = [ 0.868, -0.02 ] #-1050.45 19.5%
q11p3RmDi   =[1.431421, 0.817087, 2.028664, 0.957856, 1.974961, 0.425763] # -1081.7 32.3=p_success, old approx=95.5, new approx = 93.3 
q11p3HW10_interp_TT_interpNM = [0.6286729798152555, 0.6094902517294243, 0.5605768275595322, -0.011444792772741205, -0.00425603944107247, -0.012365080443628575] #interp_TT -1067.7 37.0%
q11p1TradXmH =  [2.4438652809143067, 1.5738661033630374]#-12623 1.0% CM140 new r_approx=43.8%
q11p3TradXmHqq =[-1.4926870960164018, -2.318859891143705, -5.881436314418812, 1.2582855789828138, 0.31574711088728313, 2.5490460457525597]  #-12293. 0.002 = gnd state prob,  0.955 = approx_ratio, CM140
q11p3TradXmH_interp_from_p1 =     [-0.6192225858511726, -0.3778830066802197, -7.7985735007499e-06, 1.573682853644343, 0.8154860817413727, -3.446929781736484e-05] #-12885.265 0.025271412390374358 1 300 
q11p3TradXmH_interp_TT_from_p1 =      [-0.37878142557194916, 8.906034487538886e-05, -0.6154068094481024, 1.57327276621201, 0.004972151376513632, -0.003760861752566373] #-13326.243 0.01261894177042601 0 312
opt_betagamma = q11p3HW10_interp_TT_interpNM
p = len(opt_betagamma)//2

if n_qubits == 5 :state_feasible = '11011'
if n_qubits == 11 :state_feasible = '11111011111'
if 0:state_feasible =None
ansatz_prog = ansatz_prog_init(prog_init, ListPauli_termsMy, n_qubits, n_destinations,   p, MyMixerHam)  

get_gnd_state_probs_and_approx_ratio = get_approx_ratio_init(ansatz_prog,prog_init,ListPauli_termsMy,SumPauli_termsMy,n_qubits,p,Adjacency_constraint,Adj_x_mixer_type_constraints,  prt_details=1,state_feasible=state_feasible)
if 0:opt_betagamma = [6.898, -0.022,0,0]
get_gnd_state_probs_and_approx_ratio(opt_betagamma,prt_details=0)

-1100.0 -825.0 -14000.0 E_min_feas, E_max_feas, min_energy_constraint_X_mixer 0.0  = min_energy_constraint_problem
0.9796865769393147  = approx_ratio_old -1077.6552346332462 -8.787412290209641e-28 -1077.655234633214


(0.37039111957123283, 0.918746307757259, 0.99999999999997)